In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/bam/Downloads/Compressed/data-2024/data.csv')

In [3]:
print(data.columns)


Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')


In [4]:
# Create aggregate features for each customer
aggregate_features = data.groupby('CustomerId').agg(
    total_transaction_amount=('Amount', 'sum'),
    average_transaction_amount=('Amount', 'mean'),
    transaction_count=('TransactionId', 'count'), 
    std_dev_transaction_amount=('Amount', 'std')  
).reset_index()


In [12]:
# Convert 'TransactionStartTime' to datetime
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])

# Extract features
data['transaction_hour'] = data['TransactionStartTime'].dt.hour
data['transaction_day'] = data['TransactionStartTime'].dt.day
data['transaction_month'] = data['TransactionStartTime'].dt.month
data['transaction_year'] = data['TransactionStartTime'].dt.year

In [17]:
print(data.columns.tolist())


['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult', 'transaction_hour', 'transaction_day', 'transaction_month', 'transaction_year', 'ProviderId_ProviderId_2', 'ProviderId_ProviderId_3', 'ProviderId_ProviderId_4', 'ProviderId_ProviderId_5', 'ProviderId_ProviderId_6', 'ProductId_ProductId_10', 'ProductId_ProductId_11', 'ProductId_ProductId_12', 'ProductId_ProductId_13', 'ProductId_ProductId_14', 'ProductId_ProductId_15', 'ProductId_ProductId_16', 'ProductId_ProductId_19', 'ProductId_ProductId_2', 'ProductId_ProductId_20', 'ProductId_ProductId_21', 'ProductId_ProductId_22', 'ProductId_ProductId_23', 'ProductId_ProductId_24', 'ProductId_ProductId_27', 'ProductId_ProductId_3', 'ProductId_ProductId_4', 'ProductId_ProductId_5', 'ProductId_ProductId_6', 'ProductId_ProductId_7', 'ProductId_ProductId_8', 'ProductId_ProductId_9', 'ProductCategory_data_bundles', 'ProductCategory_financial_services', 

In [19]:
from sklearn.preprocessing import LabelEncoder
# One-Hot Encoding for existing categorical columns
one_hot_columns = ['ChannelId_ChannelId_2', 'ChannelId_ChannelId_3', 'ChannelId_ChannelId_5']

# Check for missing columns and apply One-Hot Encoding
existing_one_hot_columns = [col for col in one_hot_columns if col in data.columns]
if existing_one_hot_columns:
    data = pd.get_dummies(data, columns=existing_one_hot_columns, drop_first=True)
    print(f"One-hot encoded columns: {existing_one_hot_columns}")
else:
    print(f"No columns found for one-hot encoding from: {one_hot_columns}")

# Label Encoding for PricingStrategy and FraudResult
label_encoder = LabelEncoder()
label_encode_columns = ['PricingStrategy', 'FraudResult']

for column in label_encode_columns:
    if column in data.columns:
        data[column] = label_encoder.fit_transform(data[column])
        print(f"Label encoded column: {column}")
    else:
        print(f"Column {column} not found for label encoding.")

# Check the transformed data
print("Transformed DataFrame:")
print(data.head())

One-hot encoded columns: ['ChannelId_ChannelId_2', 'ChannelId_ChannelId_3', 'ChannelId_ChannelId_5']
Label encoded column: PricingStrategy
Label encoded column: FraudResult
Transformed DataFrame:
         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId   Amount  Value      TransactionStartTime  PricingStrategy  \
0  CustomerId_4406   1000.0   1000 2018-11-15 02:18:49+00:00                2   
1  CustomerId_4406    -20.0     20 2018-11-15 02:19:08+00:00                2   
2  CustomerId_4683    500.0    500 2018-11-15 02:44:21+00:00                2   
3   C